# AlignedBases Incongruencies Investigation
As detailed in `issue_340_info.md` we attempted to replicate the `AlignedBases` values outputed by `dnadiff`. Unfortunately, our current code only replicates the values for some cases. In this jupyter notebook we try to determine what causes the incongruencies in `AligedBases` values given by `dnadiff` and our code. 

In this investigation, we will focus two _Streptomyces_ geomes which give us incongruent results. Data avaliable in `issue_340_test_AK/inputs/test_1`.

__What can we do?__

`dnadiff` determines which genome sequences were aligned at least once. It subsequently updates the `AlignedBases` value by considering the total length of those sequences (both aligned and unaligned regions). This total is then refined by subtracting the gaps in the alignment, as provided in the associated `.rdiff` file. These gaps, are derivered/identified by `SHOW_DIFF` program. Whre, in our implementation, we identify the total length of aligned regions (no unaligned regions, but includes overlaps if there are any). Subsequently, we find the overlapping regions and subtract them from the value. 

As we already have the information about the exact position of the aligments, we can identy the `gaps` (unaligned regions) and compare them to the ones provided in `.rdiff` file. This can help us determine was causes the diffrence in reporting diffrent number of `AligedBases`.

__Set Up__

In [14]:
from pathlib import Path
from collections import defaultdict
import intervaltree
import pandas as pd
from Bio import SeqIO

__Step 1:__ Identification of regions that are currently assumed not aligned in our code.

Here, we reuse our code which we written to get the all aligned regions (with overlaps).

In [15]:
def parse_delta(infname):
    """Parse delta files, and return all
    aligned regions (with overlaps). 

    :param infname: Path to delta file
    """

    current_ref = None
    current_qry = None

    regions_ref = defaultdict(list) #Hold a dictionary for refence regions
    regions_qry = defaultdict(list) #Hold a dictionary for query regions

    insertions_ref_positions = defaultdict(list)
    insertions_qry_positions = defaultdict(list)


    for line in [_.strip().split() for _ in infname.open("r").readlines()]:

            if line[0] == "NUCMER":  # Skip headers
                    continue
            if line[0].startswith(">"):
                current_ref = line[0].strip('>')
                current_qry = line[1]
            if len(line) == 7:
                regions_ref[current_ref].append(tuple(sorted(list([int(line[0]), int(line[1])])))) #aligned regions reference
                regions_qry[current_qry].append(tuple(sorted(list([int(line[2]), int(line[3])])))) #aligned regions qry
            
    return regions_ref

In [16]:
test_1 = parse_delta(Path("../issue_340_tests_AK/outputs_dnadiff/test_1/test_1.mdelta"))

Now, we can merge the overlapping regions with `IntervalTree` like before.

In [17]:
seq_regions_no_overlaps = defaultdict(list)

for key in test_1:
    ref_tree = intervaltree.IntervalTree.from_tuples(test_1[key])
    ref_tree.merge_overlaps(strict=False)
    for _ in sorted(ref_tree):
        seq_regions_no_overlaps[key].append(tuple([_[0], _[1]]))

Create a dictionary with all sequences and their lengths found in a genome

In [18]:
records = list(SeqIO.parse("../issue_340_tests_AK/inputs/test_1/GCF_003369795.1_ASM336979v1_genomic.fna", "fasta"))
refs = {record.id:len(record.seq) for record in records}
print(refs)

{'NZ_CP026730.1': 7787608}


Finding unaligned regions.

In [19]:

def find_non_aligned_regions(sequence_lengths, aligned_regions):
    """Find and return unalgned regions. 
    
    The returned data will be a dictionary, with the following format:
    {<sequence_id>:[(gap_start, gap_end)]}
    
    :param sequence_lengths: dictionary with sequence lengths keyed by squence id
    :aligned_regions: dictionary with aligned regions keyed by sequence id
    
    
    """
    non_aligned_regions = {}

    for sequence_id, sequence_length in sequence_lengths.items():
        aligned_ranges = aligned_regions[sequence_id]

        non_aligned = []

        if aligned_ranges[0][0] > 1:
            non_aligned.append((1, aligned_ranges[0][0] - 1))


        for i in range(1, len(aligned_ranges)):
            non_aligned.append((aligned_ranges[i - 1][1] + 1, aligned_ranges[i][0] - 1))
            

        if aligned_ranges[-1][1] < sequence_length:
            non_aligned.append((aligned_ranges[-1][1] + 1, sequence_length))

        non_aligned_regions[sequence_id] = non_aligned

    return non_aligned_regions


gaps = find_non_aligned_regions(refs, seq_regions_no_overlaps)

#Create a dataframe with each row containing information about the gap_start, gap_end and the sequence id
df = pd.DataFrame(columns=['sequence_id', 'gap_start', 'gap_end', 'method'])

for sequence_id, values in result.items():
    for start, end in values:
        df.loc[len(df)] = [sequence_id, start, end, 'IntervalTree']

Identification of gaps that do not match.

Here, we can look at the `.rdiff` file to find rows that do not match based on values in `seuqnece_id`, `gap_start` and `gap_end` columns.

In [24]:
rdiff = pd.read_csv(Path("../issue_340_tests_AK/outputs_dnadiff/test_1/test_1.rdiff"), sep='\t', names=['sequence_id', 'feature', 'gap_start', 'gap_end', 'gap_length', 'previous_seq', 'next_seq'])

In [25]:
rdiff = rdiff[(rdiff['gap_length'] >= 0) & (rdiff['feature'] != 'DUP')]

In [26]:
# Merge the DataFrames on the specified columns
merged_df = df.merge(rdiff, on=['sequence_id', 'gap_start', 'gap_end'], how='outer', indicator=True)

# Filter rows where the indicator column has the value 'left_only' or 'right_only'
rows_not_matching = merged_df.query('_merge != "both"').drop(columns=['_merge'])


rows_not_matching

,sequence_id,gap_start,gap_end,method,feature,gap_length,previous_seq,next_seq
2572,NZ_CP026730.1,4258702.0,4263070.0,IntervalTree,NaN,NaN,NaN,NaN
4321,NZ_CP026730.1,4258702.0,4263071.0,NaN,SEQ,4370.0,lcl|NZ_CP043317.1_cds_WP_194276459.1_3169,lcl|NZ_CP043317.1_cds_WP_194276455.1_3164


Through this investigation, we found that although our code identified a gap with the same start position as the one identified by show-diff, our gap end is shorter by 1 bp.

FWIW, I think looking at the delta mcoords file to identify which sequences (both reference and query) are associated with these regions and re-running the analysis strictly on them, then examining this again.